# 线性回归的简介实现

> 通过使用深度学习框架来简洁实现 线性回归模型 生成数据集

In [3]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [12]:
true_w = torch.tensor([2., -3.5])
true_b = 3.1

features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 调用框架现有的API读取数据

In [13]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch的数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train) 

In [14]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [15]:
next(iter(data_iter))

[tensor([[-0.6412,  2.0522],
         [-0.7857, -0.2912],
         [-0.0227,  0.4584],
         [-0.9630,  1.4674],
         [ 0.3448, -0.3206],
         [-1.7797, -0.3257],
         [ 0.0060, -0.5232],
         [-0.2875,  0.4021],
         [ 0.2894,  0.3898],
         [ 0.2280,  1.2617]]),
 tensor([[-5.3623],
         [ 2.5639],
         [ 1.4700],
         [-3.9722],
         [ 4.9110],
         [ 0.6994],
         [ 4.9391],
         [ 1.1110],
         [ 2.3010],
         [-0.8578]])]

### 使用框架预定义好的层

In [16]:
from torch import nn

In [17]:
net = nn.Sequential(nn.Linear(2, 1))

### 初始化模型参数 

In [19]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 计算均方误差使用 MSELoss类， 也称L2范数

In [20]:
loss = nn.MSELoss()

### 实例化SGD实例

In [21]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### 训练

In [25]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()  # 参数更新
    l = loss(net(features), labels)
    print(f"epoch {epoch + 1}, loss {l:f}")

epoch 1, loss 0.000096
epoch 2, loss 0.000096
epoch 3, loss 0.000096


In [26]:
net.parameters()

<generator object Module.parameters at 0x7f8221e184a0>